In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
# from plotly.subplots import make_subplots
import plotly.express as px
from pathlib import Path
from pprint import pprint, pformat
import os
plots_folder = 'figures'
Path(plots_folder).mkdir(parents=True, exist_ok=True)


random_seed = 65487
np.random.seed(random_seed)

In [ ]:
df_2a = pd.read_csv('data-for-figures/figure2a.csv')
df_2b = pd.read_csv('data-for-figures/figure2b.csv')
df_2c = pd.read_csv('data-for-figures/figure2c.csv')
df_2d = pd.read_csv('data-for-figures/figure2d.csv')

def process(df):
    df = df.loc[df['type']=='together', :].copy()
    df['frac'] = df['high-fidelity']/(df['high-fidelity'] + df['low-fidelity'])
    dfg = df.groupby(by=['hi-bm', 'hi-rf'], axis=0, as_index=False).agg(
    {
        'low-fidelity': ['max', 'mean'] , 
        'high-fidelity': ['max', 'mean'],
        'frac': ['mean', 'var']
    })
    rfli = sorted(dfg['hi-rf'].unique())
    bmli = sorted(dfg['hi-bm'].unique())
    frac_reshaped = dfg[('frac', 'mean')].values.reshape((len(bmli), len(rfli))).T
    return frac_reshaped, bmli, rfli

# df_2a = process(df_2a)
# df_2b = process(df_2b)
# df_2c = process(df_2c)
# df_2d = process(df_2d)


In [ ]:
for df, name in zip(
    [df_2a, df_2b, df_2c, df_2d],
    ['figure2a', 'figure2b', 'figure2c', 'figure2d']):

    frac_reshaped, bmli, rfli = process(df)
    fig = px.imshow(frac_reshaped,
                    labels=dict(x = r'$\Large{{\Delta E}, \text { housekeeping cost}}$',
                                y = r'$\Large{p, \text{ prob. of successful reproduction}}$', 
                                color=r"Relative<br>abundance"),
                    x=bmli,
                    y=rfli, 
                    origin='lower'
                   )
    fig.update_layout({
        'font': {'size': 16},
        'height': 600,
        'width': 800,
    #     'xaxis_range': [0.205, 0.34]
    }
    )    
    fig.show()
    fig.write_image(os.path.join(plots_folder, name + '.svg'))
# fig.show()
